In [10]:
import pandas as pd
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')
import torch
import numpy as np
import matplotlib.pyplot as plt

In [11]:
urlav = "https://raw.githubusercontent.com/olagit/Structure-vs-properties-of-compounds/main/data/Avalon.csv"
avalon = pd.read_csv(urlav)

In [12]:
avalon = avalon.drop(["index", "smiles"], axis=1)
avalon

,logP,fp0,fp1,fp2,fp3,fp4,fp5,fp6,fp7,fp8,...,fp502,fp503,fp504,fp505,fp506,fp507,fp508,fp509,fp510,fp511
0,-0.770,0,0,0,0,1,0,0,0,1,...,1,0,0,0,0,0,0,0,0,0
1,-3.300,0,0,0,0,0,1,0,0,1,...,0,0,1,1,0,1,0,0,0,0
2,-2.060,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,-7.870,0,0,0,0,0,0,0,1,0,...,1,0,0,0,0,0,0,0,0,0
4,-1.330,1,0,0,0,0,0,0,0,0,...,0,0,0,1,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1123,-1.710,0,0,0,0,0,0,0,1,0,...,0,0,0,1,0,0,1,0,0,0
1124,0.106,0,0,0,0,0,0,0,0,0,...,0,0,1,1,0,0,0,0,1,0
1125,-3.091,1,1,1,1,0,0,0,1,0,...,1,0,0,0,0,0,0,0,0,0
1126,-3.180,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
X = avalon.iloc[:,1:]
y = avalon.iloc[:,0]
X = torch.tensor(X.values)
Y = torch.tensor(y.values)
X.shape

torch.Size([1128, 512])

In [14]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from tqdm.notebook import tqdm, trange
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split

from typing import List, Tuple

from mldd.metrics import mae, rmse, rocauc, r_squared
from mldd.data import *

def train(X_train, y_train, X_valid, y_valid):
    hidden_size = 512
    epochs = 50
    batch_size = 64
    learning_rate = 0.0001

    model = nn.Sequential(nn.ReLU(),
                          nn.Softmax(dim=1),
                          nn.Linear(512, 1))
                          
    model.train()

    dataset = TensorDataset(torch.FloatTensor(X_train.float()), torch.FloatTensor(y_train.float().reshape(-1, 1)))
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    
    optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9)
    loss_fn = nn.MSELoss()
    for epoch in trange(1, epochs + 1, leave=False):
        for X, y in tqdm(loader, leave=False):
            model.zero_grad()
            preds = model(X)
            loss = loss_fn(preds,y)
            loss.backward()
            optimizer.step()
    return model


def predict(model, X_test, y_test):
    batch_size = 64

    dataset = TensorDataset(torch.FloatTensor(X_test.float()), torch.FloatTensor(y_test.float().reshape(-1, 1)))
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)
    
    preds_batches = []
    with torch.no_grad():
        for X, Y in tqdm(loader):
            preds = model(X)
            preds_batches.append(preds.cpu().detach().numpy())
    preds = np.concatenate(preds_batches)
    return preds

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.7, random_state=123)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.5, random_state=123)
print(torch.FloatTensor(y_test.float()))
print(y_test.type())

scores = []

for _ in range(20):#(X_train.shape[0]):
    model = train(X_train, y_train, X_valid, y_valid)

    predictions = predict(model, X_test, y_test)
    
    rmse_score = np.sqrt(np.power(y_test[:,None] - predictions, 2).mean())
    mae_score = np.abs(y_test[:,None] - predictions).mean()
    scores.append([rmse_score, mae_score])
    
#     break

scores = np.array(scores)
print('RMSE, MAE = ' + \
      ', '.join(f'{mean:.2f}±{std:.3f}' for mean, std in zip(scores.mean(axis=0), scores.std(axis=0))))

print(predictions)


tensor([ -4.5540,  -1.7900,  -7.0100,  -5.1600,  -0.9200,  -1.6000,  -0.5100,
         -4.6780,  -3.7850,  -8.0030,  -3.0100,  -1.8400,  -2.3300,   1.0200,
         -1.9500,  -3.7700,  -1.7700,  -0.4000,  -1.9500,  -0.5000,  -0.6300,
         -2.8430,  -6.1400,  -3.0800,   0.3200,  -6.8760,  -5.2400,  -8.0170,
         -4.7000,  -3.8170,  -6.5230, -11.6000,  -1.4800,  -5.8900,  -3.0820,
         -1.4580,   1.1440,  -3.9990,  -1.4700,  -3.1000,  -4.6500,  -2.4000,
         -0.6670,   0.1060,  -3.8930,  -4.2100,  -1.6400,   0.7000,  -1.5500,
         -4.4500,  -3.1500,  -2.3700,  -0.1400,  -4.3700,  -3.6700,  -4.2300,
         -2.2660,  -4.7490,  -5.0300,  -2.0000,  -3.0900,  -2.1110,  -2.4900,
         -4.8900,  -2.5700,  -6.4900,  -3.2390,  -5.1900,  -1.5000,  -4.7100,
         -5.2700,  -1.0600,  -2.5500,  -4.6300,  -2.5900,  -3.0300,  -1.9600,
          0.4500,  -2.3900,  -1.3400,  -3.7800,  -4.2100,  -3.6400,  -8.4000,
         -2.1000,  -0.2800,  -2.2400,  -1.7420,  -2.1200,  -4.26


RMSE, MAE = 2.11±0.007, 1.68±0.005
[[-2.9829502]
 [-2.9838479]
 [-2.9918025]
 [-2.9791782]
 [-2.9745789]
 [-2.9816878]
 [-2.9746058]
 [-2.980955 ]
 [-2.9815822]
 [-2.9867513]
 [-2.9839225]
 [-2.9788666]
 [-2.9822574]
 [-2.9767773]
 [-2.979495 ]
 [-2.9842124]
 [-2.9755492]
 [-2.9812891]
 [-2.9782164]
 [-2.974854 ]
 [-2.9802513]
 [-2.9850671]
 [-2.9903734]
 [-2.9779203]
 [-2.9787195]
 [-2.9870548]
 [-2.9884417]
 [-2.989594 ]
 [-2.980802 ]
 [-2.9850194]
 [-2.9810658]
 [-2.9900765]
 [-2.9799526]
 [-2.991273 ]
 [-2.9806654]
 [-2.979423 ]
 [-2.9794998]
 [-2.9843872]
 [-2.9780176]
 [-2.9809628]
 [-2.9838035]
 [-2.979063 ]
 [-2.979945 ]
 [-2.9760177]
 [-2.9855561]
 [-2.9850695]
 [-2.9829187]
 [-2.9735088]
 [-2.9734893]
 [-2.9822664]
 [-2.9818478]
 [-2.9775767]
 [-2.976158 ]
 [-2.98238  ]
 [-2.9812088]
 [-2.9833941]
 [-2.9788806]
 [-2.9858584]
 [-2.9859388]
 [-2.9839225]
 [-2.9841044]
 [-2.977981 ]
 [-2.9822962]
 [-2.9897177]
 [-2.978723 ]
 [-2.9822543]
 [-2.9809504]
 [-2.9896846]
 [-2.9814358